In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
root_path = '/content/drive/MyDrive/Data/dataset'
os.chdir(root_path)

In [ ]:
import pandas as pd
import numpy as np
import csv
import re
import os
import shutil
from sklearn.preprocessing import OneHotEncoder

In [ ]:
!pip install contractions
import contractions

     |████████████████████████████████| 287 kB 5.3 MB/s 
     |████████████████████████████████| 106 kB 55.1 MB/s 


In [ ]:
!pip install unidecode
import unidecode

     |████████████████████████████████| 235 kB 5.1 MB/s 


In [ ]:
data_raw = pd.read_csv("train.csv", escapechar = "\\", quoting = csv.QUOTE_NONE, error_bad_lines=False, na_filter=False)

In [ ]:
data_raw = data_raw[data_raw['BROWSE_NODE_ID']<=100]

In [ ]:
label = data_raw['BROWSE_NODE_ID'].values
data = data_raw.drop(columns=['BROWSE_NODE_ID','BULLET_POINTS']).values

In [ ]:
data.shape, label.shape

((2903024,), (2903024,))

Get data with 1000 labels, 100 sample each label, whose label frequency is >= 100 

In [ ]:
import collections
counter = collections.Counter(data_raw['BROWSE_NODE_ID'].values)

Total labels with more than 100 samples

In [ ]:
count = 0
for x, y in counter.items():
  if(y>100):
    count += 1
print(count)

2512


In [ ]:
remove = []
tempdict = {}
for i in range(2903024):
  if(counter[data_raw.iloc[i]['BROWSE_NODE_ID']]>=100):
    if(data_raw.iloc[i]['BROWSE_NODE_ID'] in tempdict and tempdict[data_raw.iloc[i]['BROWSE_NODE_ID']]<100):
      tempdict[data_raw.iloc[i]['BROWSE_NODE_ID']] += 1
    elif(data_raw.iloc[i]['BROWSE_NODE_ID'] not in tempdict and len(tempdict)<1000):
      tempdict[data_raw.iloc[i]['BROWSE_NODE_ID']] = 1
    else:
      remove.append(i)
  else:
    remove.append(i)
data = data_raw.drop(remove)

Get 100 samples of 50 labels for training

In [ ]:
remove = []
tempdict = {}
for i in range(100000-1, -1, -1):
  if(data_raw.iloc[i]['BROWSE_NODE_ID'] in tempdict and tempdict[data_raw.iloc[i]['BROWSE_NODE_ID']]<100):
    tempdict[data_raw.iloc[i]['BROWSE_NODE_ID']] += 1
  elif(data_raw.iloc[i]['BROWSE_NODE_ID'] not in tempdict and len(tempdict)<50):
    tempdict[data_raw.iloc[i]['BROWSE_NODE_ID']] = 1
  else:
    remove.append(i)

Get 20 samples of 50 labels for testing

In [ ]:
remove = []
tempdict2 = {}
for i in range(100000-1, -1, -1):
  if(data_raw.iloc[i]['BROWSE_NODE_ID'] in tempdict):
    if(data_raw.iloc[i]['BROWSE_NODE_ID'] in tempdict2 and tempdict2[data_raw.iloc[i]['BROWSE_NODE_ID']]<20):
      tempdict2[data_raw.iloc[i]['BROWSE_NODE_ID']] += 1
    elif(data_raw.iloc[i]['BROWSE_NODE_ID'] not in tempdict2):
      tempdict2[data_raw.iloc[i]['BROWSE_NODE_ID']] = 1
    else:
      remove.append(i)
  else:
    remove.append(i)

In [ ]:
data = data_raw.drop([data_raw.index[i] for i in remove])

In [ ]:
import pickle
data.to_pickle('data_val.pkl')

Data Processing for training

In [ ]:
label = data_raw['BROWSE_NODE_ID'].values
data = data_raw['BULLET_POINTS'].values
data = data_raw.drop(columns=['BROWSE_NODE_ID','BRAND']).values

In [ ]:
def standardize_brand(data):
  data = unidecode.unidecode(data)
  data = re.sub(r"[^A-Za-z0-9]", "", data)
  data = data.lower()
  return data

Get total labels for each brand

In [ ]:
brands = {}
for i in range(2903024):
  data[i] = standardize_brand(data[i])
  if(data[i] not in brands):
    brands[data[i]] = set()
  brands[data[i]].add(label[i])

In [ ]:
def standardize_text(data):
  data = ' '.join(data)
  data = contractions.fix(data)
  data = re.sub(r"[^A-Za-z]", " ", data)
  data = data.lower()
  return data

In [ ]:
data_processed = []
for d in data:
  data_processed.append(standardize_text(d))
data_processed = np.array(data_processed)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
label_t = label.reshape(-1,1)

In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)
label_encoded = onehot_encoder.fit_transform(label_t)

In [ ]:
label_encoded.shape

(5000, 50)